In [44]:
import requests
from bs4 import BeautifulSoup
import csv


def fetch_attraction_info(num_links):
    URL = "https://www.tripadvisor.fr/Attractions-g187147-Activities-oa0-Paris_Ile_de_France.html"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver les liens des attractions
    attraction_link_elements = soup.select('a[target="_blank"]')
    # Supprimer les éléments comme #REVIEWS des liens
    cleaned_links = ['https://www.tripadvisor.fr' + elem['href'].split('#')[0] for elem in attraction_link_elements if 'Attraction_Review' in elem['href']]

    # Utilisez un ensemble pour éliminer les doublons
    attraction_links = list(set(cleaned_links))

    activite_data = []

    
    for link in attraction_links[:num_links]:
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # nom
        activite_name = soup.select_one('h1[data-automation="mainH1"]').text.strip()

        # ouverture
        open_element = soup.select_one('.EFKKt')
        open = open_element.text.strip() if open_element else 'Ouverture non disponible'


        # Récupérer le prix de l'attraction
        price_container = soup.select_one('.f.k.O.ncFvv')
        prix_element = price_container.select_one('.biGQs._P.fiohW.uuBRH') if price_container else None
        prix = prix_element.text.strip() if prix_element else 'Prix non disponible'

        # durée
        time_container = soup.select_one('._c')
        time_element = time_container.select_one('.biGQs._P.pZUbB.KxBGd') if time_container else None
        time = time_element.text.strip() if time_element else 'Prix non disponible'

        # Récupérer l'adresse de l'attraction
        adress_container = soup.select_one('.UikNM._G.B-._S._W._T.c.G_.wSSLS.wnNQG.raEkE')
        adress_element = adress_container.select_one('.biGQs._P.XWJSj.Wb') if adress_container else None
        adress = adress_element.text.strip() if adress_element else 'Adresse non disponible'

        # note
        note = soup.select_one('.biGQs._P.fiohW.hzzSG.uuBRH').text.strip()
        

        activite_data.append({
            'Nom': activite_name,
            'open': open,
            'prix': prix,
            'time':time,
            'adress': adress,
            'note': note
        })


    return activite_data


act = fetch_attraction_info(40)



def save_to_csv(data, filename):
    keys = data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

save_to_csv(act, 'activities.csv')
